<center>
<img src="https://avatars.mds.yandex.net/i?id=08831819d2b500822a90327ea7aee923ef8a5322-9154878-images-thumbs&n=13" height="250">

# Андан на экономе: домашнее задание 3
</center>


> Bicycle bicycle bicycle  <br> I want to ride my bicycle <br> I want to ride my bike <br> I want to ride my bicycle <br>  I want to ride it where I like

$\qquad$ $\qquad$ $\qquad$ $\qquad$  **[Bicycle Race (Queen)](https://www.youtube.com/watch?v=r2lT4oE4DxE)**

**ФИО:**

**Твой любимый аник:**

## Формат сдачи

Сам ноутбук называйте в формате hw-02-USERNAME.ipynb, где USERNAME — ваши фамилия и имя на латинице. Задание сдаётся в anytask.

In [3]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

# Спрос на велосипеды

В этой домашке мы будем работать с данными о спросе на велосипеды. Мы научимся его предсказывать.

In [4]:
df = pd.read_csv('bikes_rent.csv')

print(df.shape)

df.head( )

FileNotFoundError: [Errno 2] No such file or directory: 'bikes_rent.csv'

__Описание переменных:__

* `season`  1 - весна, 2 - лето, 3 - осень, 4 - зима
* `yr`  0 - 2011, 1 - 2012
* `mnth`  от 1 до 12
* `holiday`  0 - нет праздника, 1 - есть праздник
* `weekday`  от 0 до 6
* `workingday`  0 - нерабочий день, 1 - рабочий день
* `weathersit`  оценка благоприятности погоды от 1 (чистый, ясный день) до 4 (ливень, туман)
* `temp`  температура в Цельсиях
* `atemp`  температура по ощущениям в Цельсиях
* `hum`  влажность
* `windspeed(mph)`  скорость ветра в милях в час
* `windspeed(ms)`  скорость ветра в метрах в секунду
* `cnt`  количество арендованных велосипедов (это целевой признак, его мы будем предсказывать)

In [ ]:
# Функция для того, чтобы на одной картинке смотреть те графики с прогнозами:

def plot_yreal_ypred(y_test, y_train, y_test_hat, y_train_hat):

    margin = 0.1 # отступ на границах
    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.scatter(y_train, y_train_hat, color="red", alpha=0.5)
    plt.xlabel('Истинные значения')
    plt.ylabel('Предсказанные значения')
    plt.axis('equal')
    plt.axis('square')
    train_min = min(y_train)
    train_max = max(y_train)
    plt.xlim(train_min - margin, train_max + margin)
    plt.ylim(train_min - margin, train_max + margin)
    plt.plot([-10000, 10000], [-10000, 10000])
    plt.title('Train set', fontsize=20)

    plt.subplot(122)
    plt.scatter(y_test, y_test_hat, color="red", alpha=0.5)
    plt.xlabel('Истинные значения')
    plt.ylabel('Предсказанные значения')
    plt.axis('equal')
    plt.axis('square')
    test_min = min(y_test)
    test_max = max(y_test)
    plt.xlim(test_min - margin, test_max + margin)
    plt.ylim(test_min - margin, test_max + margin)
    plt.plot([-10000, 10000], [-10000, 10000])
    plt.title('Test set', fontsize=20)
    
    pass

## 1. EDA

__а) [5 баллов]__ Проверьте, есть ли в данных пропуски

In [ ]:
missing_values = df.isnull().sum()

print(missing_values)

__Ответ:__ В данных нет пропусков.

__б) [5 баллов]__ Составьте список из категориальных переменных и список из действительных переменных.

In [ ]:
var_cat = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']

var_real = ['temp', 'atemp', 'hum', 'windspeed(mph)', 'windspeed(ms)']

target = 'cnt'

print('Список категориальных переменных: ', var_cat)
print('Список действительных переменных: ', var_real)

__в) [5 баллов]__ Нарисуйте гистограмму для целевого признака. Что можете сказать о его хвосте? Надо ли предпринять какие-нибудь манипуляции, связанные с борьбой с выбросами?

In [ ]:
sns.histplot(df[target], bins=20, kde=True)
plt.title('Распределение количества арендованных велосипедов')
plt.xlabel('Количество арендованных велосипедов')
plt.ylabel('Частота')

plt.show()

__Ответ:__ Выбросы на графике имеются ("хвосты" можно наблюдать в точках 2750 и 7750 по оси Ox), но апроксимирующая линия идёт довольно плавно, согласно Гауссовскому распределению.

__г) [5 баллов]__ Давайте посмотрим на то, как переменные коррелируют с таргетом. Для этого постройте соответствующую облака рассеивания.

In [ ]:
plt.figure(figsize=(16, 12))

for var in var_cat:
    plt.subplot(4, 4, var_cat.index(var) + 1)
    sns.scatterplot(x=var, y=target, data=df, alpha=0.5)
    plt.title('Scatterplot of {} vs target'.format(var))
    plt.xlabel(var)
    plt.ylabel(target)

for var in var_real:
    plt.subplot(4, 4, var_real.index(var) + 1 + len(var_cat))
    sns.scatterplot(x=var, y=target, data=df, alpha=0.5)
    plt.title('Scatterplot of {} vs target'.format(var))
    plt.xlabel(var)
    plt.ylabel(target)

plt.tight_layout()

plt.show()

__д) [5 баллов]__ Каков характер зависимости числа прокатов от месяца? От каких признаков спрос зависит линейно?

__Ответ на 1 вопрос:__  зависимость числа прокатов от месяца года напоминает полиномиальную, симметричную зависимость.

Для того, чтобы ответить на второй вопрос, построим матрицу корреляции и по итоговым значениям корреляции определим от каких признаков спрос зависит линейно.
Высокие по модулю значения коэффициента корреляции указывают на сильную линейную зависимость между переменными.

In [ ]:
correlation_matrix = df.corr()

correlation_with_target = correlation_matrix['cnt'].sort_values(ascending=False)

print(correlation_with_target)

__Ответ на 2 вопрос:__ Наивысший по модулю коэффициент от числа прокатов (нашего таргета) наблюдается с переменными atemp и temp. То есть можно предположить, что число прокатов линейно зависит от температуры воздуха.

__е) [5 баллов]__ Постройте гистограммы для всех непрерывных переменных. Что можете сказать о выбросах?

In [ ]:
plt.figure(figsize=(16, 10))

for i, var in enumerate(var_real):
    plt.subplot(2, 3, i+1)
    plt.hist(df[var], bins=30, edgecolor='black')
    plt.title('Histogram of {}'.format(var))
    plt.xlabel(var)
    plt.ylabel('Frequency')

plt.tight_layout()

plt.show()

__Ответ:__ Выбросы можно заметить на гистограммах температур (temp в значениях 20, 24, 30; atemp в значениях 25, 26, 31, 32).

__ж) [5 баллов]__ Постройте между непрерывными признаками корреляционную матрицу. Нарисуйте её. Какие признаки, скорее всего, неинформативны? Почему они на ваш взгляд такие?

In [ ]:
correlation_matrix_var_real = df[var_real].corr()

sns.heatmap(correlation_matrix_var_real, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix of continuous variables')

plt.show()

__Ответ:__ Temp и Atemp сильно коррелируют между собой, так как "температура по ощущениям" (atemp) сильно зависит от фактической температуры (temp), между Windspeed (mph) и Windspeed (ms) также прослеживается сильная корреляция, поскольку это одно и то же измерение, только в разных единицах. На основе этого, мы можем сделать вывод, что во избежание избыточности можно оставить лишь 1 переменную температуры и 1 переменную, отвечающую за скорость ветра. То есть 2 переменные из 5 оказались неинформативными.

__з) [5 баллов]__ Постройте для всех категориальных переменных _countplot_. Что интересного вы видите в распределеии этих признаков?  Встречаются ли слишком обширные категории, в которых очень маленькие части нам придётся объединять в категорию "другое"?

In [ ]:
plt.figure(figsize=(16, 10))

for i, var in enumerate(var_cat):
    plt.subplot(3, 3, i+1)
    sns.countplot(x=var, data=df)
    plt.title('Countplot of {}'.format(var))
    plt.xlabel(var)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()

__Ответ:__  Распределение переменных season, yr, mnth, weekday равномерно.
Из интересного: праздников гораздо меньше, чем не праздников (логично); рабочих дней в два раза больше, чем не рабочих (что тоже абсолютно логично); стоит обратить внимание на переменную weathlist (оценка благоприятности погоды от 1 (чистый, ясный день) до 4 (ливень, туман)): по распределению видно, что при неблагоприятной погоде никто ни разу не брал велосипед в прокат за всё время сбора статистики,оценка "3" благоприятности тоже крайне непопулярна среди велопользователей.
Предполагаю, что оценки 3 и 4 благоприятности погоды можно было бы объединить в категорию "другое", но оценка 4 настолько непопулярна, что её даже нет в собранной за несколько лет статистике.

__и) [5 баллов]__  Сделайте OHE-кодирование. Сколько свежих OHE-признаков у нас появилось?

In [ ]:
df_ohe = pd.get_dummies(df[var_cat])
num_ohe = df_ohe.shape[1]

print("Количество свежих OHE-признаков:", num_ohe)

__к) [5 баллов]__   Некоторые из наших переменных могут вносить в спрос на велики квадратичный вклад. Например, температура.

Сначала, когда температура возрастает, люди выходят кататься охотнее, а потом, когда становится жарко, спрос резко падает. Так возникает параболическая зависимость.

Создайте для каждой действительной переменной из данных дополнительную переменную: её квадрат.

In [ ]:
for i in var_real:
    df[i + '_squared'] = df[i] ** 2

__л) [5 баллов]__  Попробуйте придумать ещё какую-нибудь новую фичу (дополнительную переменную) для прогнозирования. Опишите какая за вашей фичёй стоит интуиция и почему это должно сработать при моделировании.

Моя идея - создание новой переменной index_of_comf, которая будет оценивать комфортность для велосипедиста, переменная зависит от температуры воздуха, влажности и скорости ветра, а также оценки благоприятности погоды. Чем выше значение переменной index_of_comf, тем более комфортные условия для велосипедиста.

In [ ]:
# Веса:

w_temp = 0.4
w_hum = 0.3
w_windspeed = 0.2
w_weathersit = 0.1

df['index_of_comf'] = (w_temp * df['temp'] +
                       w_hum * df['hum'] +
                       w_windspeed * df['windspeed(ms)'] +
                       w_weathersit * df['weathersit'])


Температура воздуха: Эта переменная имеет наибольший вес, потому что температура играет ключевую роль в комфортном вождении на велосипеде.

Влажность: Высокая влажность может сделать жару более невыносимой, в то время как низкая влажность может вызвать ощущение сухости.

Скорость ветра: Скорость ветра имеет меньший вес, потому что она влияет на комфорт в меньшей степени, чем температура и влажность. Однако сильный ветер может усложнять вождение, вследствии чего индекс комфорта понизится.

Оценка благоприятности погоды: Этот фактор представляет собой категориальную оценку общей благоприятности погоды. В отличие от температуры, влажности и скорости ветра, его влияние на комфортность может быть менее прямым.

In [ ]:
df.head( )

## 2. Обучение модели

__а) [10 баллов]__  Разбейте выборку на тренировочную и тестовую.

Разбиваю выборку в соотношении 80/20. 80% на обучение - тренировочная часть и 20% на тестирование.

In [ ]:
from sklearn.model_selection import train_test_split

x = df.drop('cnt', axis=1)
y = df['cnt']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

__б) [10 баллов]__ Возьмите RMSE в качестве метрики качества. Для этого нужно написать свою функцию. Постройте наивный прогноз (среднее) и найдите для него RMSE.

In [ ]:
def calculate_rmse(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

naive_prediction = np.mean(y_train)
naive_predictions_test = np.full_like(y_test, fill_value=naive_prediction)
rmse_naive = calculate_rmse(y_test, naive_predictions_test)

print("RMSE для наивного прогноза:", rmse_naive)

__в) [5 баллов]__ Запустите код с разбиением выборки на тренировочную и тестовую, подсчётом наивного прогноза и RMSE по нему несколько раз. НЕ НАДО НИЧЕГО КОПИРОВАТЬ. ПРОСТО ПЕРЕЗАПУСТИТЕ КОД ТРИ РАЗА. Меняется ли RMSE? Почему оно меняется? К чему плохому это может привести при обучении? Как это обычно исправляют?

__Ответ:__  RMSE не меняется, так как изначально мной был зафиксирован seed для случайного разбиения данных 'random_state=42'. Изменяющееся RMSE указывает на нестабильность оценки качества модели, что может усложнить процесс сравнения и выбора модели.

Чтобы стабилизировать процесс оценки модели и получать воспроизводимые результаты, обычно фиксируют seed для генератора случайных чисел. Также можно использовать cross-validation, то есть перекрестную проверку, которая делит данные на несколько частей и обучает модель на каждой части, а затем усредняет результаты, чтобы получить более стабильную оценку производительности модели.

__г) [10 баллов]__ Оцените линейную регрессию. Насколько удалось улучшить качество прогнозирования в сравнении с наивным прогнозом?  Визуализируйте прогнозы функцией `plot_yreal_ypred`.

Что вы видите на этой визуализации? Проинтерпретируйте картинку.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("RMSE для наивного прогноза:", rmse_naive)
print("RMSE для линейной регрессии на тренировочной выборке:", rmse_train)
print("RMSE для линейной регрессии на тестовой выборке:", rmse_test)

__Ответ:__ Модель линейной регрессии повысила качество прогнозирования по сравнению с наивным прогнозом. Разница между RMSE для линейной регрессии на тренировочной и на тестовой выборках имеют довольно близкие значения в сравнении с RMSE для наивного прогноза.

In [ ]:
# Визуализация прогнозов:

def plot_yreal_ypred(y_test, y_train, y_test_hat, y_train_hat):
    margin = 0.1
    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.scatter(y_train, y_train_hat, color="red", alpha=0.5)
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.axis('equal')
    plt.axis('square')
    train_min = min(y_train)
    train_max = max(y_train)
    plt.xlim(train_min - margin, train_max + margin)
    plt.ylim(train_min - margin, train_max + margin)
    plt.plot([-10000, 10000], [-10000, 10000])
    plt.title('Train set', fontsize=20)

    plt.subplot(122)
    plt.scatter(y_test, y_test_hat, color="red", alpha=0.5)
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.axis('equal')
    plt.axis('square')
    test_min = min(y_test)
    test_max = max(y_test)
    plt.xlim(test_min - margin, test_max + margin)
    plt.ylim(test_min - margin, test_max + margin)
    plt.plot([-10000, 10000], [-10000, 10000])
    plt.title('Test set', fontsize=20)
    plt.show()

plot_yreal_ypred(y_test, y_train, y_test_pred, y_train_pred)

__Ответ:__ Как на тестовой, так и на тренировочной выборках, точки лежат достаточно близко к диагонали, что означает хорошее качество прогнозов.
Если же все точки лежали бы на диагонали, то это был бы идеальный случай прогнозирования.
Однако возможна ситуация, когда точки сильно отклоняются от диагонали, что указывает на ошибки в модели.

__д) [10 баллов]__ Какими получились значения перед коэффициентами у вашей модели?  Как думаете, почему такое произошло?


In [ ]:
coefficients = model.coef_

print("Коэффициенты:")
for feature, coef in zip(x_train.columns, coefficients):
    print(f"{feature}: {coef}")

__Ответ:__

Если коэффициент положительный, то это означает, что с увеличением значения признака, целевой признак также увеличивается. Например, temp имеет довольно высокий положительный коэффициент, это значит, что с повышением температуры увеличивается и количество арендованных велосипедов.
Переменные temp и atemp, а также windspeed(mph) и windspeed(ms) имеют разные знаки. Так, вероятно, происходит из-за того, что возникает мультиколлинеарность (два или более признаков сильно коррелируют между собой, что может затруднять интерпретацию коэффициентов модели линейной регрессии).
Решение данной проблемы - удаление одного из сильно коррелирующих признаков из каждой пары.

## 3. Бонус

__a) [2 балла]__ Обучите Lasso-регрессию. В качестве силы регуляризации возьмите $\alpha = 40$.

* Каким получилось качество прогноза?
* Что в модели произошло с коэффициентами?  

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=40)
lasso.fit(x_train, y_train)

y_train_pred_lasso = lasso.predict(x_train)
y_test_pred_lasso = lasso.predict(x_test)

rmse_train_lasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
rmse_test_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))

print('RMSE для Lasso-регрессии на тренировочной выборке: ', rmse_train_lasso)
print('RMSE для Lasso-регрессии на тестовой выборке: ', rmse_test_lasso)

In [ ]:
coefficients_lasso = lasso.coef_

print("Коэффициенты:")
for feature, coef in zip(x_train.columns, coefficients_lasso):
    print(f"{feature}: {coef}")

__Ответ:__   Качество прогноза получилось высокое, разница между RMSE на тренировочной и тестовой выборках небольшая.
Некоторые из коэффициентов стали равны нулю, в том числе и переменные с высокой корреляцией по одному из каждой пары тоже обнулились.
Другие же коэффициенты (неравные нулю) уменьшились.

__б) [0 баллов]__  Изучите код в этом пункте и прочитайте выводы. Подумайте об этом.

В случае линейных моделей, обычно, используют два вида регуляризации: Ridge и Lasso. Первая прибавляет к MSE квадраты коэффицентов, а вторая модули. Давайте посмотрим какая между ними разница.

In [ ]:
from sklearn.linear_model import Lasso, Ridge  # подгружаем модели

Обучим много-много моделей с разными силами регуляризации $\alpha$ и посмотрим что получится.

In [ ]:
alphas = np.arange(1, 100, 2)   # задали диапазон для силы регуляризатора

# матрицы для весов перед коэффициентами (число регрессоров)*(число признаков)
coefs_lasso = np.zeros((alphas.shape[0], x.shape[1]))
coefs_ridge = np.zeros((alphas.shape[0], x.shape[1]))

features = x.columns

i = 0
for alph in alphas:
    rg = Ridge(alpha=alph)  # для каждого alph обучаем модель
    ls = Lasso(alpha=alph)
    rg.fit(x,y)
    ls.fit(x,y)
    coefs_ridge[i] = rg.coef_  # и запоминаем коэффициенты
    coefs_lasso[i] = ls.coef_
    i += 1

В отличие от L2-регуляризации (Ridge), L1 (Lasso) обнуляет веса при некоторых признаках. Давайте пронаблюдаем, как меняются веса при увеличении коэффициента регуляризации $\alpha$ (в лекции коэффициент при регуляризаторе мог быть обозначен другой буквой).

In [ ]:
plt.figure(figsize=(8, 5))
for coef, feature in zip(coefs_lasso.T, features):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3))
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Lasso")

plt.figure(figsize=(8, 5))
for coef, feature in zip(coefs_ridge.T, features):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3))
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Ridge")

Картинки вышли красивыми. Обратите внимание, что регуляризатор Lasso агрессивнее уменьшает веса. Подумайте с чем это связано. Если делать `alpha` очень большим, веса зануляются. В случае Ridge такого не происходит. Веса становятся всё ближе к нулю, но зануления не происходит.  Из-за того, что Lasso зануляет некоторые признаки, его можно использовать для отбора самых важных признаков. Дальше будем работать именно с ним.

Итак, мы видим, что при изменении `alpha` модель по-разному подбирает коэффициенты признаков. Нам нужно выбрать наилучшее `alpha`. Для этого, во-первых, нам нужна метрика качества. Будем использовать в качестве метрики сам оптимизируемый функционал метода наименьших квадратов, то есть MSE (Mean Square Error).

Во-вторых, нужно понять, на каких данных эту метрику считать. Нельзя выбирать `alpha` по значению MSE на обучающей выборке, потому что тогда мы не сможем оценить, как модель будет делать предсказания на новых для нее данных. Если мы выберем одно разбиение выборки на обучающую и тестовую, то настроимся на конкретные "новые" данные, и вновь можем переобучиться.

Именно такую настройку вы наблюдали выше, когда запускали код с наивным прогнозом много раз подряд. Поэтому будем делать несколько разбиений выборки, на каждом пробовать разные значения `alpha`, а затем усреднять MSE. Удобнее всего делать такие разбиения кросс-валидацией, то есть разделить выборку на $K$ частей, или блоков, и каждый раз брать одну из них как тестовую, а из оставшихся блоков составлять обучающую выборку.

Подбирать параметр `alpha` в `sklearn` совсем просто: для этого есть `GridSearchCV`. Мы уже сталкивались с примером применения этой функции на семинаре. Попробуйте с помощью неё перебрать список из альф и подыскать оптимальное. Параметр cv отвечает за то, на сколько частей делится выборка. Поставьте `cv=5`. Также укажите параметр `scoring = neg_mean_squared_error`. Это задаст функцию потерь, на которую будет ориентироваться `GridSearchCV` при переборе. Назовите переменную, в которой будет находиться перебор __grid_cv_lasso.__

__в) [2 балла]__ Подберите для Lasso-регрессии с помощью поиска по решётке оптимальное значение $\alpha$.  Каким оказалось это значение?


In [ ]:
from sklearn.model_selection import GridSearchCV
alpha_range = np.logspace(-4, 4, 50)

lasso = Lasso()
param_grid = {'alpha': alpha_range}
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)

best_alpha = grid_search.best_params_['alpha']
print('Оптимальное значение альфа: ', best_alpha)

Оптимальное значение альфа:  5.428675439323859, округлим до 5.4

__г) [2 балла]__ Каким получается качество прогноза для модели с оптимальным значением $\alpha$?

In [ ]:
best_lasso = grid_search.best_estimator_
y_pred = best_lasso.predict(x_test)
rmse = np.sqrt(((y_pred - y_test) ** 2).mean())
print('RMSE на тестовой выборке: ', rmse)

__д) [2 балла]__ Наконец, как принято в анализе данных, давайте проинтерпретируем результат. Проинтерпретируйте последнюю обученую модель. У каких признаков наибольшие положительные коэфициенты? У каких наибольшие отрицательные? Логично ли утверждать, что чем больще/меньше эти признаки, тем выше/ниже спрос на велосипеды? Какие коэффициенты занулились?

In [ ]:
coef_df = pd.DataFrame({'Переменная': features, 'Коэффициент': coefficients})
coef_df = coef_df.sort_values(by='Коэффициент', ascending=False)
print(coef_df)

print("Наибольшие положительные коэффициенты:")
print(coef_df.head())

print("Наибольшие отрицательные коэффициенты:")
print(coef_df.tail())

print("Зануленные коэффициенты:")
print(coef_df[coef_df['Коэффициент'] == 0])

__Ответ:__  Нет, в данном случае не логично утверждать, что чем больше/меньше эти признаки, тем выше/ниже спрос на велосипеды.

__е) [2 балла]__ В прошлой домашке, когда мы придумывали фичи, мы предположили, что спрос на велосипеды зависит от скорости ветра квадратично, по параболе. Подтвердилось ли это предположение?

__Ответ:__  Коэффициент windspeed(ms)_squared = 1.129945e+12, что подтверждает предположение о том, что спрос на велосипеды может зависеть от скорости ветра нелинейно, по форме, например, параболы.
Такое присутствие квадратичного члена указывает на то, что увеличение скорости ветра может как положительно, так и отрицательно влиять на спрос на велосипеды, в зависимости от уровня скорости ветра.